# 批量预测

1. 需要指定要批量预测的样本。

   1. 自己手动指定。
   2. 通过`mydir`参数循环找到指定目录下的所有图片数据。
   
2. 指定Onekey训练好的目录路径，对应到`config_path参数`，精确到`viz`目录。
> 例如：path2yours/20220802/inception_v3_bb/viz

In [9]:
import os
from onekey_algo import get_param_in_cwd
from glob import glob
import pandas as pd

POSSIBLE_IMG = ['.jpg', '.jpeg', '.png', '.bmp']

#data_root = get_param_in_cwd('data_root')
data_root = r'D:\bianyun_onekey'
config_path = os.path.join(data_root, 'models', 'resnet18', 'viz')
train = pd.read_csv(os.path.join(data_root, 'split_info', 'train-RND-0.txt'), sep='\t', header=None, names=['fname', 'label'])
test = pd.read_csv(os.path.join(data_root, 'split_info', 'val-RND-0.txt'), sep='\t', header=None, names=['fname', 'label'])
already = set([os.path.basename(s) for s in set(train['fname']) | set(test['fname'])])
#all_samples = glob(os.path.join(data_root, 'patches', '*/*.jpg'))
all_samples = glob(r'E:\bian_data\*/*.jpg')
samples = [s for s in all_samples if os.path.basename(s) not in already]
print(len(samples))


5543


In [4]:
import json
from onekey_algo.classification.eval_classification import init as clf_init
from onekey_algo.classification.eval_classification import inference_dataloader as clf_inference

model, transformer, labels, device = clf_init(config_path=config_path)
results = clf_inference(samples, model, transformer, labels, device, num_workers=12, batch_size=64)

r = pd.DataFrame(results, columns=['fname', 'prob', 'pred'])
r['gt'] = 1
r['prob'] = r['prob'].map(lambda x: json.loads(x)['1']).round(decimals=2)
r.to_csv(os.path.join(data_root, 'models', 'resnet18', 'viz', 'predictions_bian_data.csv'), index=False)
r[['group']] = r[['fname']].applymap(lambda x: x.split('_')[0])




# 移动特征

In [8]:
r.to_csv(os.path.join(data_root, 'models', 'resnet18', 'viz', 'predictions.csv'), index=False)

In [9]:
import shutil
import os

features_dir = os.path.join(data_root, 'models/SimpleViT/viz/features')
for f in os.listdir(features_dir):
    fn = os.path.splitext(f)[0]
    shutil.move(os.path.join(features_dir, f), os.path.join(features_dir, f"{fn.split('-')[0]}.csv"))

In [10]:
label_data = pd.read_csv(os.path.join(data_root, 'split_info/label-RND-0.csv'), dtype={'ID': str})

for _, row in label_data.iterrows():
    group = row['group']
    save2 = os.path.join(features_dir, group)
    os.makedirs(save2, exist_ok=True)
    shutil.move(os.path.join(features_dir, f"{row['ID']}.csv"), save2)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\bianyun_onekey\\models/SimpleViT/viz/features\\1750735.csv'